In [74]:
"""This file is a test playground for new scripts and functions."""

%load_ext autoreload
%autoreload 2

###### USER SETTINGS ######
SERIAL_ACTIVE = False # if False, just runs the algorithm without sending to HV switches

###### INITIALIZATIONS ######
import cv2
import time
import serial
import numpy as np
import visual_haptic_utils.haptic_funcs as haptic_funcs # my custom file
import visual_haptic_utils.USB_writer as USB_writer # my custom file
import matplotlib.pyplot as plt

###### MAIN ######

# initialize camera and gesture model:
frame_rate = 24
generator = haptic_funcs.IntensityGenerator(total_time=5, frame_rate=frame_rate)
haptic_map = haptic_funcs.HapticMap()

output_list = []
output_list.extend(generator.ramp(direction=1))
output_list.extend(generator.ramp(direction=-1))

duty_array_list, period_array_list = haptic_map.linear_map_sequence(output_list,
                                                           freq_range=(0,200),
                                                           duty_range=(0.05,0.5))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
duty_array_list[10]

(30,)

In [4]:
packet_list = USB_writer.make_packet_sequence(duty_array_list, period_array_list)

In [9]:
for packet in packet_list[50]:
    print(packet)

b'P\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00T\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00'
b'P\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00T\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00'
b'P\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x05\x00\x00\x00T\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x16\x00\x00\x00'


In [77]:
###### USER SETTINGS ######
FILENAME = "algo_input_data/datakoi_output.txt"
VIDEONAME = "algo_input_videos/video_koi.mp4"
SERIAL_ACTIVE = True # if False, just runs the algorithm without sending to HV switches
COM_A = "COM9" # port for MINI switches 1-10
COM_B = "COM15" # port for MINI switches 11-20
COM_C = "COM16" # port for MINI swiches 21-28

###### INITIALIZATIONS ######
import cv2
import time
import visual_haptic_utils.haptic_funcs as haptic_funcs # my custom file
import visual_haptic_utils.USB_writer as USB_writer # my custom file
from numpy import genfromtxt

###### MAIN ######
# Load data
data=genfromtxt(FILENAME,delimiter=',')[:,0:28]

In [80]:
data.shape

(1796, 28)